## Import Libaries and Packages

In [121]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.svm import SVR
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn.feature_selection import SelectKBest, f_regression

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning);

## Import and Investigate the Data

In [122]:
df_train = pd.read_csv('/Users/ben/Desktop/DSI_GA_Materials/project_5/data/df_2000_to_2017_pollution_renewables.csv')

In [123]:
df_test = pd.read_csv('/Users/ben/Desktop/DSI_GA_Materials/project_5/data/df_2018_to_2020_pollution_renewables.csv')

In [124]:
df_train.dtypes

State                                                                object
Year                                                                  int64
Month                                                                 int64
O3 Mean                                                             float64
O3 1st Max Value                                                    float64
O3 1st Max Hour                                                     float64
O3 AQI                                                              float64
CO Mean                                                             float64
CO 1st Max Value                                                    float64
CO 1st Max Hour                                                     float64
CO AQI                                                              float64
SO2 Mean                                                            float64
SO2 1st Max Value                                                   float64
SO2 1st Max 

In [125]:
df_train.shape

(6063, 27)

In [126]:
df_test.shape

(1536, 27)

In [127]:
df_train.columns

Index(['State', 'Year', 'Month', 'O3 Mean', 'O3 1st Max Value',
       'O3 1st Max Hour', 'O3 AQI', 'CO Mean', 'CO 1st Max Value',
       'CO 1st Max Hour', 'CO AQI', 'SO2 Mean', 'SO2 1st Max Value',
       'SO2 1st Max Hour', 'SO2 AQI', 'NO2 Mean', 'NO2 1st Max Value',
       'NO2 1st Max Hour', 'NO2 AQI',
       'Renewable energy share in the total final energy consumption (%)',
       'Electricity from fossil fuels (TWh)', 'Electricity from nuclear (TWh)',
       'Electricity from renewables (TWh)',
       'Low-carbon electricity (% electricity)',
       'Primary energy consumption per capita (kWh/person)',
       'Energy intensity level of primary energy (MJ/$2017 PPP GDP)',
       'Renewables (% equivalent primary energy)'],
      dtype='object')

In [128]:
df_train[['Year', 'Month','Electricity from renewables (TWh)','State']].head(40);

### Renewable energy amounts are by number of pollution observations, these are approximations based on the pollution amounts. Approximations make renewable, fossil fuel, energy usage uniform across states in order to get a rough estimate for totals upon the test set. 

## Modeling Section

### Define features and perform train-test split

In [129]:
df_train = pd.get_dummies(df_train, columns=['State'], prefix='State')

In [130]:
df_train

,Year,Month,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,...,State_South Carolina,State_South Dakota,State_Tennessee,State_Texas,State_Utah,State_Vermont,State_Virginia,State_Washington,State_Wisconsin,State_Wyoming
0,2000,1,0.018242,0.033239,9.945652,31.891304,1.336178,2.400000,10.130435,27.250000,...,False,False,False,False,False,False,False,False,False,False
1,2000,2,0.023883,0.042200,10.387500,44.050000,0.985815,1.772500,9.212500,20.162500,...,False,False,False,False,False,False,False,False,False,False
2,2000,3,0.027969,0.045933,10.483146,44.157303,0.690650,1.179775,7.966292,13.426966,...,False,False,False,False,False,False,False,False,False,False
3,2000,4,0.034200,0.054920,10.760000,58.040000,0.661423,1.154000,9.160000,13.220000,...,False,False,False,False,False,False,False,False,False,False
4,2000,5,0.038819,0.059276,10.379310,70.620690,0.539292,0.920690,6.086207,10.568966,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6058,2017,8,0.043358,0.052194,9.064516,49.709677,0.189534,0.209677,1.161290,2.096774,...,False,False,False,False,False,False,False,False,False,True
6059,2017,9,0.037946,0.046000,10.350000,45.400000,0.196542,0.255000,5.400000,2.700000,...,False,False,False,False,False,False,False,False,False,True
6060,2017,10,0.034492,0.041727,9.863636,38.727273,0.121151,0.150000,3.000000,1.500000,...,False,False,False,False,False,False,False,False,False,True
6061,2017,11,0.028938,0.035960,10.760000,33.320000,0.119833,0.164000,5.520000,1.640000,...,False,False,False,False,False,False,False,False,False,True


In [131]:
df_test = pd.get_dummies(df_test, columns=['State'], prefix='State')

In [132]:
df_test

,Year,Month,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,...,State_Pennsylvania,State_Rhode Island,State_South Dakota,State_Tennessee,State_Texas,State_Utah,State_Vermont,State_Virginia,State_Washington,State_Wyoming
0,2018,3,0.032459,0.046000,9.400000,42.600000,0.193611,0.260000,11.400000,2.800000,...,False,False,False,False,False,False,False,False,False,False
1,2018,4,0.038665,0.049333,10.700000,48.466667,0.194547,0.296667,6.000000,3.233333,...,False,False,False,False,False,False,False,False,False,False
2,2018,5,0.031921,0.048613,9.806452,49.967742,0.237279,0.341935,4.419355,3.806452,...,False,False,False,False,False,False,False,False,False,False
3,2018,6,0.026653,0.042759,9.448276,45.931034,0.205339,0.272414,3.344828,2.827586,...,False,False,False,False,False,False,False,False,False,False
4,2018,7,0.031020,0.047452,9.419355,46.258065,0.201449,0.261290,4.000000,2.774194,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531,2020,8,0.043341,0.049600,11.200000,47.200000,0.134654,0.160000,1.000000,1.600000,...,False,False,False,False,False,False,False,False,False,True
1532,2020,9,0.042220,0.049000,8.925926,47.629630,0.153333,0.200000,2.148148,2.111111,...,False,False,False,False,False,False,False,False,False,True
1533,2020,10,0.036024,0.042517,9.896552,39.724138,0.121264,0.193103,3.551724,2.068966,...,False,False,False,False,False,False,False,False,False,True
1534,2020,11,0.033542,0.039200,9.566667,36.233333,0.096906,0.123333,3.333333,1.233333,...,False,False,False,False,False,False,False,False,False,True


In [133]:
X_features = ['Year', 'Month', 'O3 Mean', 'O3 1st Max Value', 'O3 1st Max Hour',
               'O3 AQI', 'CO Mean', 'CO 1st Max Value', 'CO 1st Max Hour', 'CO AQI',
               'SO2 Mean', 'SO2 1st Max Value', 'SO2 1st Max Hour', 'SO2 AQI',
               'NO2 Mean', 'NO2 1st Max Value', 'NO2 1st Max Hour', 'NO2 AQI'] + list(df_train.columns[df_train.columns.str.startswith('State_')])

y_features = ['Electricity from fossil fuels (TWh)', 'Electricity from nuclear (TWh)',
              'Electricity from renewables (TWh)', 'Low-carbon electricity (% electricity)']

y_0 = y_features[0]

y_0


'Electricity from fossil fuels (TWh)'

In [134]:
# # Reverse
# # Performs poorly need more variation in Renewable Data
# y_features = ['O3 Mean', 'O3 1st Max Value', 'O3 1st Max Hour',
#                'O3 AQI', 'CO Mean', 'CO 1st Max Value', 'CO 1st Max Hour', 'CO AQI',
#                'SO2 Mean', 'SO2 1st Max Value', 'SO2 1st Max Hour', 'SO2 AQI',
#                'NO2 Mean', 'NO2 1st Max Value', 'NO2 1st Max Hour', 'NO2 AQI'] 

# X_features = ['Year', 'Month', 'Electricity from fossil fuels (TWh)', 'Electricity from nuclear (TWh)',
#               'Electricity from renewables (TWh)', 'Low-carbon electricity (% electricity)']+list(df_train.columns[df_train.columns.str.startswith('State_')])

In [135]:
X = df_train[X_features]
y = df_train[y_features]
y0 = df_train[y_0]

In [136]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [137]:
X_train0, X_test0, y_train0, y_test0 = train_test_split(X, y0, train_size=0.8, random_state=42)

### Scale Data

In [138]:
ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

In [139]:
ss0 = StandardScaler()
X_train_sc0 = ss.fit_transform(X_train0)
X_test_sc0 = ss.transform(X_test0)

### Start with Linear, Lasso, Ridge, Regressions

In [140]:
lin = MultiOutputRegressor(LinearRegression())

In [141]:
lin.fit(X_train_sc, y_train)

MultiOutputRegressor(estimator=LinearRegression())

In [142]:
lin0 = (LinearRegression())

In [143]:
lin0.fit(X_train_sc0, y_train0)

LinearRegression()

In [144]:
predictions_lin0 = lin0.predict(X_test_sc0)

In [145]:
train_score_lin0 = lin0.score(X_train_sc0, y_train0)
test_score_lin0 = lin0.score(X_test_sc0, y_test0)
print("The train score for Linear model is {}".format(train_score_lin0))
print("The test score for Linear model is {}".format(test_score_lin0))

The train score for Linear model is 0.9524479367223412
The test score for Linear model is 0.9486094558442932


In [146]:
#predict
predictions_lin = lin.predict(X_test_sc)

# Calculate scores for each variable
scores_r2 = [r2_score(y_test.iloc[:, i], predictions_lin[:, i]) for i in range(len(y_features))]
scores_mse = [mean_squared_error(y_test.iloc[:, i], predictions_lin[:, i]) for i in range(len(y_features))]

# Display scores for each variable
for i, feature in enumerate(y_features):
    print(f"R-squared score for {feature}: {scores_r2[i]} \n")
    print(f"Mean Squared Error for {feature}: {scores_mse[i]}\n ")

# Display overall scores
train_score_lin = lin.score(X_train_sc, y_train)
test_score_lin = lin.score(X_test_sc, y_test)
print("The train score for Linear model is {}".format(train_score_lin))
print("The test score for Linear model is {}".format(test_score_lin))

R-squared score for Electricity from fossil fuels (TWh): 0.9486094558442932 

Mean Squared Error for Electricity from fossil fuels (TWh): 0.35870255256117883
 
R-squared score for Electricity from nuclear (TWh): 0.9554628818822537 

Mean Squared Error for Electricity from nuclear (TWh): 0.021236781844367402
 
R-squared score for Electricity from renewables (TWh): 0.3362923264629818 

Mean Squared Error for Electricity from renewables (TWh): 0.01022581778219098
 
R-squared score for Low-carbon electricity (% electricity): 0.9335092880274302 

Mean Squared Error for Low-carbon electricity (% electricity): 3.0509427206394343e-05
 
The train score for Linear model is 0.7977409153103013
The test score for Linear model is 0.7934684880542398


#### `R^2` values in test are the same for single regression and multioutput regression. This demonstrates that a single regression is performed per variable and `MultiOutputRegressor` will be part of composite function moving forward. 

In [147]:
# Lasso
lasso = MultiOutputRegressor(Lasso())

lasso.fit(X_train_sc,y_train)

MultiOutputRegressor(estimator=Lasso())

In [148]:
predictions_lasso = lasso.predict(X_test_sc)


# Calculate scores for each variable
scores_r2_lasso = [r2_score(y_test.iloc[:, i], predictions_lasso[:, i]) for i in range(len(y_features))]
scores_mse_lasso = [mean_squared_error(y_test.iloc[:, i], predictions_lasso[:, i]) for i in range(len(y_features))]

# Display scores for each variable
for i, feature in enumerate(y_features):
    print(f"R-squared score for {feature} (Lasso): {scores_r2_lasso[i]} \n")
    print(f"Mean Squared Error for {feature} (Lasso): {scores_mse_lasso[i]} \n ")

# Display overall scores
train_score_lasso = lasso.score(X_train_sc, y_train)
test_score_lasso = lasso.score(X_test_sc, y_test)
print("The train score for Lasso model is {}".format(train_score_lasso))
print("The test score for Lasso model is {}".format(test_score_lasso))

R-squared score for Electricity from fossil fuels (TWh) (Lasso): 0.7965041484660975 

Mean Squared Error for Electricity from fossil fuels (TWh) (Lasso): 1.4203873996674883 
 
R-squared score for Electricity from nuclear (TWh) (Lasso): -0.0048639747771392905 

Mean Squared Error for Electricity from nuclear (TWh) (Lasso): 0.4791526240918315 
 
R-squared score for Electricity from renewables (TWh) (Lasso): -0.0017352130477865657 

Mean Squared Error for Electricity from renewables (TWh) (Lasso): 0.015433845596572859 
 
R-squared score for Low-carbon electricity (% electricity) (Lasso): -0.0050124687070458585 

Mean Squared Error for Low-carbon electricity (% electricity) (Lasso): 0.0004611524504081981 
 
The train score for Lasso model is 0.1993848610661758
The test score for Lasso model is 0.19622312298353145


#### Lasso will not be of any help here

In [149]:
# Ridge
ridge = MultiOutputRegressor(Ridge())

ridge.fit(X_train_sc,y_train)

MultiOutputRegressor(estimator=Ridge())

In [150]:
# Predictions
predictions_ridge = ridge.predict(X_test_sc)

# Calculate scores for each variable
scores_r2_ridge = [r2_score(y_test.iloc[:, i], predictions_ridge[:, i]) for i in range(len(y_features))]
scores_mse_ridge = [mean_squared_error(y_test.iloc[:, i], predictions_ridge[:, i]) for i in range(len(y_features))]

# Display scores for each variable
for i, feature in enumerate(y_features):
    print(f"R-squared score for {feature} (Ridge): {scores_r2_ridge[i]} \n")
    print(f"Mean Squared Error for {feature} (Ridge): {scores_mse_ridge[i]} \n")

# Display overall scores
train_score_ridge = ridge.score(X_train_sc, y_train)
test_score_ridge = ridge.score(X_test_sc, y_test)
print("The train score for Ridge model is {}".format(train_score_ridge))
print("The test score for Ridge model is {}".format(test_score_ridge))

R-squared score for Electricity from fossil fuels (TWh) (Ridge): 0.9487710849730477 

Mean Squared Error for Electricity from fossil fuels (TWh) (Ridge): 0.35757439207942193 

R-squared score for Electricity from nuclear (TWh) (Ridge): 0.955482430468516 

Mean Squared Error for Electricity from nuclear (TWh) (Ridge): 0.021227460427101007 

R-squared score for Electricity from renewables (TWh) (Ridge): 0.33689337196548474 

Mean Squared Error for Electricity from renewables (TWh) (Ridge): 0.010216557407431943 

R-squared score for Low-carbon electricity (% electricity) (Ridge): 0.9336127704861777 

Mean Squared Error for Low-carbon electricity (% electricity) (Ridge): 3.046194402492985e-05 

The train score for Ridge model is 0.797705271695851
The test score for Ridge model is 0.7936899144733066


## Polynomial Features

In [151]:
poly = PolynomialFeatures(degree=2,include_bias=False)

In [152]:
X_train_sc = poly.fit_transform(X_train_sc)

X_test_sc = poly.transform(X_test_sc)

In [153]:
lin.fit(X_train_sc, y_train)

KeyboardInterrupt: 

In [ ]:
ridge.fit(X_train_sc,y_train)

In [ ]:
#predict
predictions_lin = lin.predict(X_test_sc)

# Calculate scores for each variable
scores_r2 = [r2_score(y_test.iloc[:, i], predictions_lin[:, i]) for i in range(len(y_features))]
scores_mse = [mean_squared_error(y_test.iloc[:, i], predictions_lin[:, i]) for i in range(len(y_features))]

# Display scores for each variable
for i, feature in enumerate(y_features):
    print(f"R-squared score for {feature}: {scores_r2[i]} \n")
    print(f"Mean Squared Error for {feature}: {scores_mse[i]}\n ")

# Display overall scores
train_score_lin = lin.score(X_train_sc, y_train)
test_score_lin = lin.score(X_test_sc, y_test)
print("The train score for Lin model is {}".format(train_score_lin))
print("The test score for Lin model is {}".format(test_score_lin))

#### Lasso will not be of any help here

In [ ]:
# Predictions
predictions_ridge = ridge.predict(X_test_sc)

# Calculate scores for each variable
scores_r2_ridge = [r2_score(y_test.iloc[:, i], predictions_ridge[:, i]) for i in range(len(y_features))]
scores_mse_ridge = [mean_squared_error(y_test.iloc[:, i], predictions_ridge[:, i]) for i in range(len(y_features))]

# Display scores for each variable
for i, feature in enumerate(y_features):
    print(f"R-squared score for {feature} (Ridge): {scores_r2_ridge[i]} \n")
    print(f"Mean Squared Error for {feature} (Ridge): {scores_mse_ridge[i]} \n")

# Display overall scores
train_score_ridge = ridge.score(X_train_sc, y_train)
test_score_ridge = ridge.score(X_test_sc, y_test)
print("The train score for Ridge model is {}".format(train_score_ridge))
print("The test score for Ridge model is {}".format(test_score_ridge))

## KNN Regression 

In [ ]:
# KNeighborsRegressor()
knn = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=15))

In [ ]:
ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

In [ ]:
knn.fit(X_train,y_train)

In [ ]:
#predict
predictions_knn = knn.predict(X_test_sc)

# Calculate scores for each variable
scores_r2 = [r2_score(y_test.iloc[:, i], predictions_knn[:, i]) for i in range(len(y_features))]
scores_mse = [mean_squared_error(y_test.iloc[:, i], predictions_knn[:, i]) for i in range(len(y_features))]

# Display scores for each variable
for i, feature in enumerate(y_features):
    print(f"R-squared score for {feature}: {scores_r2[i]} \n")
    print(f"Mean Squared Error for {feature}: {scores_mse[i]}\n ")

# Display overall scores
train_score_knn = knn.score(X_train_sc, y_train)
test_score_knn = knn.score(X_test_sc, y_test)
print("The train score for KNN model is {}".format(train_score_knn))
print("The test score for KNN model is {}".format(test_score_knn))

In [ ]:
# Define the parameter grid to search
param_grid = {
    'estimator__n_neighbors': [5, 10, 15, 20, 25],
    'estimator__weights': ['uniform', 'distance'],
    'estimator__p': [1, 2, 3]  # Experiment with different values for p
}

# Create the KNN model with MultiOutputRegressor
knn = MultiOutputRegressor(KNeighborsRegressor())

# Create the GridSearchCV object
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train_sc, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Use the best model for predictions
best_knn = grid_search.best_estimator_
predictions_knn = best_knn.predict(X_test_sc)

In [ ]:
# Calculate scores for each variable
scores_r2 = [r2_score(y_test.iloc[:, i], predictions_knn[:, i]) for i in range(len(y_features))]
scores_mse = [mean_squared_error(y_test.iloc[:, i], predictions_knn[:, i]) for i in range(len(y_features))]

# Display scores for each variable
for i, feature in enumerate(y_features):
    print(f"R-squared score for {feature}: {scores_r2[i]} \n")
    print(f"Mean Squared Error for {feature}: {scores_mse[i]}\n ")

# Display overall scores
train_score_knn = best_knn.score(X_train_sc, y_train)
test_score_knn = best_knn.score(X_test_sc, y_test)
print("The train score for the best KNN model is {}".format(train_score_knn))
print("The test score for the best KNN model is {}".format(test_score_knn))

### More Models

#### Decision Tree Regressor

In [ ]:
dtr = MultiOutputRegressor(DecisionTreeRegressor(random_state=42))

In [ ]:
dtr.fit(X_train_sc, y_train)

In [ ]:
#predict
predictions_dtr = dtr.predict(X_test_sc)

# Calculate scores for each variable
scores_r2 = [r2_score(y_test.iloc[:, i], predictions_dtr[:, i]) for i in range(len(y_features))]
scores_mse = [mean_squared_error(y_test.iloc[:, i], predictions_dtr[:, i]) for i in range(len(y_features))]

# Display scores for each variable
for i, feature in enumerate(y_features):
    print(f"R-squared score for {feature}: {scores_r2[i]} \n")
    print(f"Mean Squared Error for {feature}: {scores_mse[i]}\n ")

# Display overall scores
train_score_dtr = dtr.score(X_train_sc, y_train)
test_score_dtr = dtr.score(X_test_sc, y_test)
print("The train score for DTR model is {}".format(train_score_dtr))
print("The test score for DTR model is {}".format(test_score_dtr))

In [ ]:
# Create a Decision Tree Regressor model with MultiOutputRegressor
dtr = MultiOutputRegressor(DecisionTreeRegressor())

# Define the parameter grid to search
param_grid = {
    'estimator__max_depth': [None, 5, 10, 15],
    'estimator__min_samples_split': [2, 5, 10],
    'estimator__min_samples_leaf': [1, 2, 4]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(dtr, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train_sc, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Use the best model for predictions
best_dtr = grid_search.best_estimator_
predictions_dtr = best_dtr.predict(X_test_sc)

# Calculate and display scores for each variable
scores_r2 = [r2_score(y_test.iloc[:, i], predictions_dtr[:, i]) for i in range(len(y_features))]
scores_mse = [mean_squared_error(y_test.iloc[:, i], predictions_dtr[:, i]) for i in range(len(y_features))]

# Display scores for each variable
for i, feature in enumerate(y_features):
    print(f"R-squared score for {feature}: {scores_r2[i]} \n")
    print(f"Mean Squared Error for {feature}: {scores_mse[i]}\n ")

# Display overall scores
train_score_dtr = best_dtr.score(X_train_sc, y_train)
test_score_dtr = best_dtr.score(X_test_sc, y_test)
print("The train score for the best DTR model is {}".format(train_score_dtr))
print("The test score for the best DTR model is {}".format(test_score_dtr))

#### Bagging Tree

In [ ]:
# Create a BaggingRegressor model with MultiOutputRegressor
br = MultiOutputRegressor(BaggingRegressor())

# Define the parameter grid to search
param_grid = {
    'estimator__n_estimators': [10, 50, 100],
    'estimator__max_samples': [0.5, 0.7, 1.0],
    'estimator__max_features': [0.5, 0.7, 1.0]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(br, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train_sc, y_train)

# Get the best parameters
best_params_br = grid_search.best_params_
print("Best Hyperparameters:", best_params_br)

# Use the best model for predictions
best_br = grid_search.best_estimator_
predictions_br = best_br.predict(X_test_sc)

# Calculate and display scores for each variable
scores_r2_br = [r2_score(y_test.iloc[:, i], predictions_br[:, i]) for i in range(len(y_features))]
scores_mse_br = [mean_squared_error(y_test.iloc[:, i], predictions_br[:, i]) for i in range(len(y_features))]

# Display scores for each variable
for i, feature in enumerate(y_features):
    print(f"R-squared score for {feature}: {scores_r2_br[i]} \n")
    print(f"Mean Squared Error for {feature}: {scores_mse_br[i]}\n ")

# Display overall scores
train_score_br = best_br.score(X_train_sc, y_train)
test_score_br = best_br.score(X_test_sc, y_test)
print("The train score for the best Bagging Regressor model is {}".format(train_score_br))
print("The test score for the best Bagging Regressor model is {}".format(test_score_br))

#### Random Forest

In [ ]:
# Create a RandomForestRegressor model with MultiOutputRegressor
rf = MultiOutputRegressor(RandomForestRegressor(random_state=42))

# Define the parameter grid to search
param_grid = {
    'estimator__n_estimators': [10, 25, 50, 100],
    'estimator__max_depth': [None, 1, 2, 5],
    'estimator__min_samples_split': [1, 2, 5],
    'estimator__min_samples_leaf': [1, 2],
    'estimator__max_features': ['auto', 'sqrt']
}

# Create the GridSearchCV object
grid_search_rf = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search to the data
grid_search_rf.fit(X_train_sc, y_train)

# Get the best parameters
best_params_rf = grid_search_rf.best_params_
print("Best Hyperparameters:", best_params_rf)

# Use the best model for predictions
best_rf = grid_search_rf.best_estimator_
predictions_rf = best_rf.predict(X_test_sc)

# Calculate and display scores for each variable
scores_r2_rf = [r2_score(y_test.iloc[:, i], predictions_rf[:, i]) for i in range(len(y_features))]
scores_mse_rf = [mean_squared_error(y_test.iloc[:, i], predictions_rf[:, i]) for i in range(len(y_features))]

# Display scores for each variable
for i, feature in enumerate(y_features):
    print(f"R-squared score for {feature}: {scores_r2_rf[i]} \n")
    print(f"Mean Squared Error for {feature}: {scores_mse_rf[i]}\n ")

# Display overall scores
train_score_rf = best_rf.score(X_train_sc, y_train)
test_score_rf = best_rf.score(X_test_sc, y_test)
print("The train score for the best RandomForest model is {}".format(train_score_rf))
print("The test score for the best RandomForest model is {}".format(test_score_rf))

#### ADA Boost

In [ ]:
# Create an AdaBoostRegressor model with MultiOutputRegressor
abr = MultiOutputRegressor(AdaBoostRegressor(random_state=42))

# Define the parameter grid to search
param_grid = {
    'estimator__n_estimators': [50, 100, 200],
    'estimator__learning_rate': [0.001, 0.01, 0.1, 1.0],
    'estimator__loss': ['linear', 'square', 'exponential']
}

# Create the GridSearchCV object
grid_search_abr = GridSearchCV(abr, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search to the data
grid_search_abr.fit(X_train_sc, y_train)

# Get the best parameters
best_params_abr = grid_search_abr.best_params_
print("Best Hyperparameters:", best_params_abr)

# Use the best model for predictions
best_abr = grid_search_abr.best_estimator_
predictions_abr = best_abr.predict(X_test_sc)

# Calculate and display scores for each variable
scores_r2_abr = [r2_score(y_test.iloc[:, i], predictions_abr[:, i]) for i in range(len(y_features))]
scores_mse_abr = [mean_squared_error(y_test.iloc[:, i], predictions_abr[:, i]) for i in range(len(y_features))]

# Display scores for each variable
for i, feature in enumerate(y_features):
    print(f"R-squared score for {feature}: {scores_r2_abr[i]} \n")
    print(f"Mean Squared Error for {feature}: {scores_mse_abr[i]}\n ")

# Display overall scores
train_score_abr = best_abr.score(X_train_sc, y_train)
test_score_abr = best_abr.score(X_test_sc, y_test)
print("The train score for the best AdaBoostRegressor model is {}".format(train_score_abr))
print("The test score for the best AdaBoostRegressor model is {}".format(test_score_abr))

#### Support Vector Machine

In [ ]:
# Create an SVR model with MultiOutputRegressor
svr = MultiOutputRegressor(SVR())

# Define the parameter grid to search
param_grid = {
    'estimator__C': [0.001, 0.01, 0.1, 1, 10],
    'estimator__kernel': ['linear', 'rbf', 'poly'],
    'estimator__degree': [2, 3],
    'estimator__epsilon': [0.1, 0.2, 0.5]
}

# Create the GridSearchCV object
grid_search_svr = GridSearchCV(svr, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search to the data
grid_search_svr.fit(X_train_sc, y_train)

# Get the best parameters
best_params_svr = grid_search_svr.best_params_
print("Best Hyperparameters:", best_params_svr)

# Use the best model for predictions
best_svr = grid_search_svr.best_estimator_
predictions_svr = best_svr.predict(X_test_sc)

# Calculate and display scores for each variable
scores_r2_svr = [r2_score(y_test.iloc[:, i], predictions_svr[:, i]) for i in range(len(y_features))]
scores_mse_svr = [mean_squared_error(y_test.iloc[:, i], predictions_svr[:, i]) for i in range(len(y_features))]

# Display scores for each variable
for i, feature in enumerate(y_features):
    print(f"R-squared score for {feature}: {scores_r2_svr[i]} \n")
    print(f"Mean Squared Error for {feature}: {scores_mse_svr[i]}\n ")

# Display overall scores
train_score_svr = best_svr.score(X_train_sc, y_train)
test_score_svr = best_svr.score(X_test_sc, y_test)
print("The train score for the best SVR model is {}".format(train_score_svr))
print("The test score for the best SVR model is {}".format(test_score_svr))